In [13]:
from datasets import load_dataset, disable_caching
import os

disable_caching()

In [18]:
data_files = {}
train_file = "data/train_clevr_obj.json"
validation_file = "data/dev_clevr_obj.json"
cache_dir = "cache"
if train_file is not None:
    data_files["train"] = train_file
    extension = train_file.split(".")[-1]
if validation_file is not None:
    data_files["validation"] = validation_file
    extension = validation_file.split(".")[-1]
dataset = load_dataset(extension, data_files=data_files, cache_dir=cache_dir)

Found cached dataset json (/home/khangln/DRIVE_H/WORK/llms/vision/contrastive_img_txt/cache/json/default-ab1f67df85342d82/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96)
100%|██████████| 2/2 [00:00<00:00, 248.76it/s]


In [19]:
img_dir = "clevr_img"


def add_full_img_path_and_caption(example):
    example[
        "caption"
    ] = f"A photo of a {example['color']} {example['material']} {example['shape']}"
    example["img"] = os.path.join(img_dir, example["image"])
    return example

In [20]:
dataset = dataset.map(add_full_img_path_and_caption)

In [22]:
dataset["validation"][0]

{'color': 'green',
 'material': 'rubber',
 'shape': 'sphere',
 'image': 'img_800.png',
 'caption': 'A photo of a green rubber sphere',
 'img': 'clevr_img/img_800.png'}

In [2]:
import json
import os

with open("data/test_unseen_shape.json") as f:
    test_data = [json.loads(line) for line in f]
img_dir = "clevr_images/unseen_shape"

for d in test_data:
    d["image_path"] = os.path.join(img_dir, d["image"])
    d["label"] = f"{d['color']} {d['material']} {d['shape']}"

In [5]:
from torch.utils.data import DataLoader

t = DataLoader(test_data, batch_size=8)

In [7]:
for a in t:
    print(a)
    break

{'color': ['blue', 'red', 'yellow', 'green', 'red', 'blue', 'yellow', 'green'], 'material': ['rubber', 'metal', 'metal', 'metal', 'metal', 'rubber', 'rubber', 'metal'], 'shape': ['donut', 'cone', 'donut', 'cone', 'donut', 'diamond', 'donut', 'cone'], 'image': ['img_0.png', 'img_1.png', 'img_2.png', 'img_3.png', 'img_4.png', 'img_5.png', 'img_6.png', 'img_7.png'], 'image_path': ['clevr_images/unseen_shape/img_0.png', 'clevr_images/unseen_shape/img_1.png', 'clevr_images/unseen_shape/img_2.png', 'clevr_images/unseen_shape/img_3.png', 'clevr_images/unseen_shape/img_4.png', 'clevr_images/unseen_shape/img_5.png', 'clevr_images/unseen_shape/img_6.png', 'clevr_images/unseen_shape/img_7.png'], 'label': ['blue rubber donut', 'red metal cone', 'yellow metal donut', 'green metal cone', 'red metal donut', 'blue rubber diamond', 'yellow rubber donut', 'green metal cone']}


In [1]:
from transformers import (
    VisionTextDualEncoderModel,
    VisionTextDualEncoderProcessor,
    AutoTokenizer,
    AutoImageProcessor,
)

model = VisionTextDualEncoderModel.from_vision_text_pretrained(
    "google/vit-base-patch16-224", "xlm-roberta-base", cache_dir="cache"
)

tokenizer = AutoTokenizer.from_pretrained(
    "xlm-roberta-base", cache_dir="cache")
image_processor = AutoImageProcessor.from_pretrained(
    "google/vit-base-patch16-224", cache_dir="cache"
)
processor = VisionTextDualEncoderProcessor(image_processor, tokenizer)

# save the model and processor
model.save_pretrained("vit-b-xlmr-b")
processor.save_pretrained("vit-b-xlmr-b")

/home/khangln/mambaforge/envs/my/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of the model checkpoint at google/vit-base-patch16-224 were not used when initializing ViTModel: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing ViTModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ViTModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ViTModel were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized: [

In [24]:
from typing import List
from PIL import Image
import requests
from transformers import (
    VisionTextDualEncoderModel,
    VisionTextDualEncoderProcessor,
    AutoImageProcessor,
    AutoTokenizer,
    AutoProcessor,
    CLIPModel,
    OwlViTProcessor,
    OwlViTModel,
)
import torch
import json
import os
from torch.utils.data import DataLoader
from tqdm import tqdm
import logging
from collections import defaultdict
from datetime import datetime

device = torch.device(
    "cuda") if torch.cuda.is_available() else torch.device("cpu")

MODEL = "models/vit-b-p16-224-roberta-b"

tokenizer = AutoTokenizer.from_pretrained(MODEL)
image_processor = AutoImageProcessor.from_pretrained(MODEL)
processor = VisionTextDualEncoderProcessor(image_processor, tokenizer)
model = VisionTextDualEncoderModel.from_pretrained(MODEL)
model.to(device)

EVAL_JSON = "data/test.json"
EVAL_IMG_DIR = "clevr_images/normal"

with open(EVAL_JSON) as f:
    test_data = [json.loads(line) for line in f]

for d in test_data:
    d["image_path"] = os.path.join(EVAL_IMG_DIR, d["image"])
    d["label"] = f"{d['color']} {d['material']} {d['shape']}"


@torch.no_grad()
def get_image_embeddings(model, images: List[Image.Image]):
    inputs = processor(text=[""], images=images, return_tensors="pt", padding=True).to(
        device
    )
    outputs = model(**inputs)
    return outputs.image_embeds


@torch.no_grad()
def get_text_embeddings(model, texts: List[str]):
    inputs = processor(
        text=texts, images=[Image.new("RGB", (2, 2))], return_tensors="pt", padding=True
    ).to(device)
    outputs = model(**inputs)
    return outputs.text_embeds


data_loader = DataLoader(test_data[:1000], batch_size=10)

In [18]:
image_embeddings = None
for batch in tqdm(data_loader):
    images = [Image.open(img_path).convert("RGB") for img_path in batch["image_path"]]
    batch_image_embeddings = get_image_embeddings(model, images)
    if image_embeddings is None:
        image_embeddings = batch_image_embeddings
    else:
        image_embeddings = torch.cat((image_embeddings, batch_image_embeddings), dim=0)

100%|██████████| 100/100 [00:11<00:00,  9.09it/s]


In [36]:
def retrieve_img(image_embeddings, text, top_k=10):
    text_embedding = get_text_embeddings(model, [text])
    logits_per_text = torch.matmul(text_embedding, image_embeddings.t())
    return torch.topk(logits_per_text[0], k=top_k).indices

In [37]:
retrieve_img(image_embeddings, "metal object")

tensor([388, 516, 687, 957, 767,  27, 777, 432, 555, 966], device='cuda:0')

In [31]:
logits_per_text = torch.matmul(query_embedding, image_embeddings.t())

In [35]:
torch.topk(logits_per_text[0], 10).indices

tensor([388, 516, 687, 957, 767,  27, 777, 432, 555, 966], device='cuda:0')

In [11]:
a = torch.empty(1)
x = torch.randn(2, 3)
x
torch.cat((a, x), 0)

RuntimeError: Tensors must have same number of dimensions: got 1 and 2

In [23]:
Image.new("RGB", (1, 1))

In [4]:
import os

In [1]:
from datasets import load_dataset

dataset = load_dataset(
    'json',
    data_files='data/dev.json',
    cache_dir='cache'
)

/home/khangln/mambaforge/envs/my/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Extracting data files: 100%|██████████| 1/1 [00:00<00:00, 80.89it/s]


Dataset json downloaded and prepared to /home/khangln/DRIVE_H/WORK/vision/contrastive_img_txt/cache/json/default-f7eb3a1a40683786/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96. Subsequent calls will reuse this data.


100%|██████████| 1/1 [00:00<00:00, 158.45it/s]


In [5]:
def add_full_img_path_and_caption(example):
    color = example["color"]
    material = example["material"]
    shape = example["shape"]
    caption = [
        f"The material is {material}",
        f"The color is {color}",
        f"The shape is {shape}",
    ]
    example['caption'] = caption
    example['image_path'] = os.path.join(
        'clevr_images', example["image"]
    )
    return example


dataset = dataset.map(add_full_img_path_and_caption)

In [9]:
dataset["train"]["caption"]

[['The material is metal', 'The color is blue', 'The shape is cylinder'],
 ['The material is rubber', 'The color is blue', 'The shape is cylinder'],
 ['The material is rubber', 'The color is gray', 'The shape is sphere'],
 ['The material is rubber', 'The color is yellow', 'The shape is sphere'],
 ['The material is rubber', 'The color is red', 'The shape is sphere'],
 ['The material is rubber', 'The color is green', 'The shape is cube'],
 ['The material is rubber', 'The color is red', 'The shape is cube'],
 ['The material is metal', 'The color is gray', 'The shape is cylinder'],
 ['The material is metal', 'The color is green', 'The shape is sphere'],
 ['The material is metal', 'The color is red', 'The shape is sphere'],
 ['The material is metal', 'The color is yellow', 'The shape is sphere'],
 ['The material is rubber', 'The color is green', 'The shape is cube'],
 ['The material is rubber', 'The color is yellow', 'The shape is sphere'],
 ['The material is rubber', 'The color is blue', '